<a href="https://colab.research.google.com/github/argalusmp/CH2-PS_Recommendation-System/blob/mi/Recommender_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q scann

import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_recommenders as tfrs
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from typing import Dict, Text

In [ ]:
user_ds = pd.read_csv("drive/MyDrive/Capstone Project/user_ds.csv")
event_ds= pd.read_csv("drive/MyDrive/Capstone Project/event_ds.csv")

df_user = pd.DataFrame(user_ds)
df_event = pd.DataFrame(event_ds)

print(len(df_user))
print(len(df_event))

1045
5599


In [ ]:
df_user

,Uid,Volunteer Name,Age,Gender,Skill 1,Skill 2 (Additional),Location,Type of Organization
0,U1,Mia Davis,27,Female,Dog training,Animal behavior modification,Solo,Pet and Animal Service
1,U2,Amelia Adams,28,Female,Pet grooming,Photography,Ngawi,Healthcare
2,U3,Jessica Thompson,29,Female,Animal shelter volunteering,Dog training,Ngawi,Healthcare
3,U4,Harper Lewis,26,Female,Mentoring,Community outreach,Solo,IT
4,U5,Mia Johnson,33,Female,Nursing,Medical assistance,Ngawi,Healthcare
...,...,...,...,...,...,...,...,...
1040,U1051,Ava Turner,26,Female,Environmental advocacy,Conservation,Tangerang,Environmental
1041,U1052,Noah Walker,32,Female,Public speaking,Advocacy,Tangerang,Event Organizer
1042,U1053,Mia Hernandez,26,Female,Mentoring,Community outreach,Bandung,IT
1043,U1054,Ethan Roberts,31,Female,Teaching,Computer literacy,Tangerang,Social


In [ ]:
df_event

,Event_id,Category,Domisili,Age,Qualifications 1,Qualifications 2,Qualifications 3
0,E1,Pet and Animal Service,Solo,>18,Veterinary assistance,Pet adoption,Veterinary medicine
1,E2,Pet and Animal Service,Jawa Barat,All,Veterinary medicine,Pet adoption,Dog walking
2,E3,Pet and Animal Service,Yogyakarta,All,Animal shelter volunteering,Animal care,Animal rescue
3,E4,Pet and Animal Service,Indonesia,18-24,Animal rescue,Animal behavior modification,Animal behavior modification
4,E5,Pet and Animal Service,Bandung,18-24,Animal rescue,Pet adoption support,Animal behavior modification
...,...,...,...,...,...,...,...
5594,E5595,Social,Tangerang,17-30,Career guidance,Research,Digital marketing
5595,E5596,Social,Kalimantan Selatan,17-30,Fundraising,Research,Community outreach
5596,E5597,Social,Jakarta,>20,Graphic design,Sports coaching,Medical research
5597,E5598,Social,Bali,16-38,Fundraising,Leadership training,Digital marketing


In [ ]:
# df_user = df_user.drop(['Age', 'Uid'], axis=1)
# df_event = df_event.drop('Age', axis=1)

In [ ]:
df_user = tf.data.Dataset.from_tensor_slices(dict(df_user))
df_event = tf.data.Dataset.from_tensor_slices(dict(df_event))

In [ ]:
user = df_user.map(lambda x: {
    "skills": [x["Skill 1"], x["Skill 2 (Additional)"]],
    "user_id": x["Uid"],
    "location": x["Location"],
    "category": x["Type of Organization"]
})

event = df_event.map(lambda x: {
    "skills": [x["Qualifications 1"], x["Qualifications 2"], x["Qualifications 3"]],
    "event_id": x["Event_id"],
    "location": x["Domisili"],
    "category": x["Category"]
})

In [ ]:
unique_skills = np.unique(np.concatenate(list(user.batch(1000).map(lambda x: x["skills"]))))
unique_location = np.unique(np.concatenate(list(event.batch(1000).map(lambda x: x["location"]))))
unique_category = np.unique(np.concatenate(list(event.batch(1000).map(lambda x: x["category"]))))

In [ ]:
tf.random.set_seed(42)

# Split between train and tests sets, as before.
shuffled = user.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(4000)
test = shuffled.skip(4000).take(1000)

embedding_dimension = 32

user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_skills, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_skills) + 1, embedding_dimension)
])

event_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_skills, mask_token=None),
  tf.keras.layers.Embedding(len(unique_skills) + 1, embedding_dimension)
])

In [ ]:
task = tfrs.tasks.Retrieval(
  metrics=tfrs.metrics.FactorizedTopK(
  candidates=event.batch(128).map(lambda x: event_model(x['skills']))
))

In [ ]:
class VolunteerModel(tfrs.Model):

  def __init__(self, user_model, event_model):
    super().__init__()
    self.event_model: tf.keras.Model = event_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["skills"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_event_embeddings = self.event_model(features["skills"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_event_embeddings)

In [ ]:
model = VolunteerModel(user_model, event_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(100).batch(1000).cache()
cached_test = test.batch(1000).cache()

In [ ]:
model.fit(cached_train, epochs=3)

Epoch 1/3


ValueError: ignored

In [ ]:
model.evaluate(cached_test, return_dict=True)

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((event.batch(100), event.batch(100).map(model.event_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['skills', 'event_id', 'location', 'category'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


AttributeError: ignored